In [1]:
import utilities as util
import difflib
from scipy import spatial
from scipy import array
from scipy.stats import mode
import time
import numpy as np
import reedsolo
from collections import OrderedDict

Data from Bas' file

In [2]:
with open("sequenced.txt", 'r') as f:
    dropletlist = [line.rstrip('\n') for line in f]

Erlich et al. mention that they only collapse identical sequences. So I will attempt to do that here.


In [3]:
def collapse(sim_results):
    sequences =  {}
    for i in sim_results:
        i = int(util.DNA2Binary(i),2)
        if i not in sequences:
            sequences[i] = 1 
        else: 
            sequences[i] += 1
    return sequences

In [4]:
collapsed = collapse(dropletlist)

In [5]:
collapsed

{13955174023864270045684288714168511531034968225425962446065969219853863095803266751205061145: 1,
 13606311304690237312977564937720028698343208933079804051109860704832474786759057769783013998: 10,
 12971661136707024043359756344339990586361380409536264757495031029469870735243412900535731071: 4,
 2822203543609552941604922111392603275205634893457885592975974178171228822388586048070015026: 1,
 26188745821550745004443643922248318909934080150293298632682223450543725888766524433277598195: 5,
 5205623835555887773976614561719572346589815800398042374507101369495604867695240001953654194: 6,
 10668468541633421065438151824699918008784264955326378976419346596643404615018340142339639484: 4,
 19542417378234430447980168890484263699785650019398432881270858523411158257498091597183380642: 16,
 5956153921991084872574608119666659872368423758271821432605750913972741678190535961459433372: 1,
 18615917736029638024670726027637004277912903312881081765779249533451897468189996509895163088: 11,
 1705838396855311046

In [6]:
def HammingDistance(num1,num2):
    return bin(num1 ^ num2).count("1")

Wall time: 145 ms


In [9]:
groups={}
ungrouped = []
for i in collapsed:
    if collapsed[i] > 1: 
        groups[i] = collapsed[i]
    else:
        ungrouped.append(i)

In [12]:
def average_sequence(group):
    test = {}
    correct_sequence = ""
    for g in group:
        for i, c in enumerate(format(g, "0304b")):
            if i in test: 
                test[i].append(c)
            else:
                test[i] = [c]
    for i in range(304):
        correct_sequence += mode(test[i])[0][0]
    return correct_sequence

In [141]:
def find_best_match(group):
    current = 200
    best_sequence = ""    
    for i in group:
        distance = []
        for j in group:
            distance.append(HammingDistance(i,j))        
        if np.average(distance) < current:
            best_sequence = i
            print(distance,np.average(distance),current, best_sequence)
            current = np.average(distance
    return best_sequence


In [106]:
def CollapseGroupedSmart(grouped, ungrouped):
    collapsed = []
    for i in grouped:
        group = [i] * grouped[i]
        for j in ungrouped:
            if HammingDistance(i,j) < 4:
                group.append(j)
                ungrouped.remove(j)
        collapsed.append(group)
    return collapsed, ungrouped
def CollapseUngroupedSmart(ungrouped):
    collapsed = []
    for i in ungrouped:
        group = [i]
        ungrouped.remove(i)
        for j in ungrouped:
            if HammingDistance(i,j) < 4:
                group.append(j)
                ungrouped.remove(j)
        collapsed.append(group)
    return collapsed

In [140]:
%%time
grouped_grouped ,remaining  = CollapseGroupedSmart(groups,ungrouped)
ungrouped_grouped = CollapseUngroupedSmart(remaining)
for i in ungrouped_grouped:
    grouped_grouped.append(i)
final_grouping = ungrouped_grouped

Wall time: 36min 30s


In [154]:
final_grouping[151]

[1810857222783384978689866889834189730453449891380929714814645815127152783072666184697833525,
 1810857222785237652117663948960966866213588897906582034569296064151784104416792794772072501]

In [160]:
sequences = {}
for i in final_grouping:
    number_of_reads = len(i)
    group_sequence = util.Binary2DNA(average_sequence(i))
    sequences[group_sequence] = number_of_reads

In [161]:
with open("collapsed.txt" , "w") as file:
    for w in sorted(sequences, key=sequences.get, reverse=True):
        file.write(w + "\n")

In [166]:
for i in sequences:
    if sequences[i] > 1:
        print(i)

...


In [163]:
len(sequences[0])

KeyError: 0